In [1]:
%pip install boto3 numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import numpy as np
import pandas as pd
import json
import os
from boto3.dynamodb.types import TypeDeserializer

path = 'D:/OneDrive - Insper/Diego/artigo_monocraticas/dados_brutos'

def deserialize_dynamo_data(data):
    d = TypeDeserializer()
    new_data = {}
    for k, v in data["Item"].items():
        if isinstance(v, dict):
            dynamodb_type = list(v.keys())[0]
            value = v[dynamodb_type]
            if dynamodb_type == 'M':
                new_data[k] = {sub_k: d.deserialize({list(sub_v.keys())[0]: list(sub_v.values())[0]}) for sub_k, sub_v in value.items() if sub_v is not None}
            elif dynamodb_type == 'L':
                new_data[k] = [d.deserialize({list(sub_item.keys())[0]: list(sub_item.values())[0]}) for sub_item in value if list(sub_item.values())[0] is not None]
            else:
                new_data[k] = d.deserialize({dynamodb_type: value})
        else:
            new_data[k] = v
    return new_data


def read_data_from_file(file_path):
    data = []
    with open(file_path, 'r',encoding='UTF-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def deserialize_data_from_file(file_path):
    data = read_data_from_file(file_path)
    return [deserialize_dynamo_data(item) for item in data]

numero_processo = list()
for arquivo in os.listdir(path):
    data = deserialize_data_from_file(f'{path}/{arquivo}')
    for processo in data:
        for andamento in processo['pagina_processo']:
            if 'classe_e_num' in andamento and processo['pagina_processo']['classe_e_num'].startswith('Inq'):
                numero_processo.append(processo['pagina_processo'])
df = pd.DataFrame(numero_processo)

In [11]:
df.columns

Index(['relator', 'partes', 'numero_unico', 'andamentos',
       'detalhes_tamanho_fisico', 'classe_e_num', 'assunto',
       'data_de_protocolo', 'origem', 'numero_origem', 'orgao_origem',
       'relator_ultimo_inc'],
      dtype='object')

In [5]:
# df1 = pd.read_excel('inquerito_interesse.xlsx')
df1 = pd.read_excel('inqueritos_stf.xlsx')
df1.head()

,Data Autuação,data,ano,Título,Numerações,Etiqueta,Número de processo Único (NUP),Gênero,Classe Processual,Sigla da Classe Processual,Unidade do MPF,Ofício de Distribuição,Membro Responsável,Câmara,Assunto CNMP,Localização,processo,PGR,nomeacao,numero
0,29/11/2004,2004-11-29,2004,"29.11.2004 - INQ 2164 (STF, 2T), PGR-SEJUD",2164;STF-2164-INQ,STF-2164-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,PGR-SECRETARIA JURÍDICA E DE DOCUMENTAÇÃO,PROCURADOR-GERAL DA REPÚBLICA: CÍVEL,PAULO GUSTAVO GONET BRANCO,NaN,NaN,STF - SUPREMO TRIBUNAL FEDERAL,0,Cláudio Lemos Fonteles,1,2164
1,19/05/2005,2005-05-19,2005,"19.05.2005 - INQ 2221 (STF, PLE), PGR-SEJUD",2221;STF-2221-INQ,STF-2221-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,PGR-SECRETARIA JURÍDICA E DE DOCUMENTAÇÃO,PROCURADOR-GERAL DA REPÚBLICA: CÍVEL,PAULO GUSTAVO GONET BRANCO,NaN,NaN,STF - SUPREMO TRIBUNAL FEDERAL,0,Antonio Fernando de Souza,1,2221
2,04/10/2000,2000-10-04,2000,"04.10.2000 - INQ 1669 (STF, PLE), PGR-SEJUD",1669;STF-1669-INQ,STF-1669-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,PGR-SECRETARIA JURÍDICA E DE DOCUMENTAÇÃO,PROCURADOR-GERAL DA REPÚBLICA: CRIMINAL/STF,ANA BORGES COELHO SANTOS,NaN,NaN,STF - SUPREMO TRIBUNAL FEDERAL,0,Geraldo Brindeiro,0,1669
3,13/06/2000,2000-06-13,2000,"13.06.2000 - INQ 838 (STF, PLE), PGR-SEJUD",838;STF-838-INQ,STF-838-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,PGR-SECRETARIA JURÍDICA E DE DOCUMENTAÇÃO,PROCURADOR-GERAL DA REPÚBLICA: CRIMINAL/STF,ANA BORGES COELHO SANTOS,NaN,NaN,STF - SUPREMO TRIBUNAL FEDERAL,0,Geraldo Brindeiro,0,838
4,24/02/2003,2003-02-24,2003,"24.02.2003 - INQ 1900 (STF, PLE), PGR-SEJUD",1900;STF-1900-INQ,STF-1900-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,PGR-SECRETARIA JURÍDICA E DE DOCUMENTAÇÃO,PROCURADOR-GERAL DA REPÚBLICA: CÍVEL,ELIZETA MARIA DE PAIVA RAMOS,NaN,NaN,STF - SUPREMO TRIBUNAL FEDERAL,0,Geraldo Brindeiro,0,1900


In [15]:
df1.numero = df1.numero.astype(str)

In [16]:
# df1['numero'] = df1.Numerações.str.split(';').str[0]
# interesse = list(df1.numero)
df['numero'] = df.classe_e_num.str.split().str[1]
pd.merge(df1,df, left_on='numero', right_on='numero', how='left').to_excel('inquerito_stf_final.xlsx', index=False)

In [ ]:
import pandas as pd
import re

df = pd.read_excel('inquerito_interesse_dados_stf.xlsx')
# df.head()

,Data Autuação,data,ano,Título,Numerações,Etiqueta,Número de processo Único (NUP),Gênero,Classe Processual,Sigla da Classe Processual,...,numero_unico,andamentos,detalhes_tamanho_fisico,classe_e_num,assunto,data_de_protocolo,origem,numero_origem,orgao_origem,relator_ultimo_inc
0,29/11/2004,2004-11-29,2004,"29.11.2004 - INQ 2164 (STF, 2T), PGR-SEJUD",2164;STF-2164-INQ,STF-2164-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,...,Número Único: 0003960-90.2004.0.01.0000,"[{'data': '22/09/2006', 'nome': 'BAIXA AO ARQU...","{'Folhas': '421', 'Apensos': '0', 'Volumes': '4'}",Inq 2164,CRIME CONTRA A FÉ PÚBLICA | FALSIDADE IDEOLÓGI...,05/10/2004,DISTRITO FEDERAL,104833,SUPREMO TRIBUNAL FEDERAL,NaN
1,19/05/2005,2005-05-19,2005,"19.05.2005 - INQ 2221 (STF, PLE), PGR-SEJUD",2221;STF-2221-INQ,STF-2221-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,...,Número Único: 0001949-54.2005.0.01.0000,"[{'data': '28/11/2008', 'nome': 'Baixa ao arqu...","{'Folhas': '332', 'Apensos': '4', 'Volumes': '6'}",Inq 2221,DIREITO PROCESSUAL PENAL | Investigação Penal,13/05/2005,RORAIMA,56078,SUPREMO TRIBUNAL FEDERAL,NaN
2,04/10/2000,2000-10-04,2000,"04.10.2000 - INQ 1669 (STF, PLE), PGR-SEJUD",1669;STF-1669-INQ,STF-1669-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,...,Número Único: 0003388-76.2000.0.01.0000,"[{'data': '22/05/2001', 'nome': 'BAIXA AO ARQU...","{'Folhas': '24', 'Apensos': '1', 'Volumes': '1'}",Inq 1669,DIREITO PENAL | Crimes Previstos na Legislação...,21/09/2000,AMAPÁ,86253,SUPREMO TRIBUNAL FEDERAL,NaN
3,13/06/2000,2000-06-13,2000,"13.06.2000 - INQ 838 (STF, PLE), PGR-SEJUD",838;STF-838-INQ,STF-838-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,...,Número Único: 0000163-58.1994.0.01.0000,"[{'data': '07/11/2000', 'nome': 'BAIXA AO ARQU...","{'Folhas': '39', 'Apensos': '0', 'Volumes': '1'}",Inq 838,DIREITO PENAL | Crimes contra a Honra | Difama...,02/02/1994,SÃO PAULO,1862,SUPREMO TRIBUNAL FEDERAL,NaN
4,24/02/2003,2003-02-24,2003,"24.02.2003 - INQ 1900 (STF, PLE), PGR-SEJUD",1900;STF-1900-INQ,STF-1900-INQ,NaN,INQUÉRITO POLICIAL/TCO,INQUÉRITO,INQ,...,Número Único: 0000498-62.2003.0.01.0000,"[{'data': '06/12/2004', 'nome': 'BAIXA AO ARQU...","{'Folhas': '232', 'Apensos': '0', 'Volumes': '2'}",Inq 1900,DIREITO PENAL | Crimes contra a Honra | Calúni...,12/02/2003,AMAPÁ,52782002,JUIZ DE DIREITO,NaN


In [24]:
import ast
# df['partes'] = df['partes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['partes'] = df['partes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

partes_interesse = list()
etiqueta = list()
for _, row in df.iterrows():
    for item in row.partes:
        if isinstance(item, list):
            tipo = item[0]
            if re.search(r'invest|qte', tipo, re.IGNORECASE):
                partes_interesse.append(item[1])
                etiqueta.append(row.Etiqueta)
# pd.DataFrame({'Etiqueta': etiqueta, 'parte_interesse': partes_interesse}).to_excel('partes_interesse_stf.xlsx', index=False)
df1 = pd.DataFrame({'Etiqueta': etiqueta, 'parte_interesse': partes_interesse})
pd.merge(df, df1, on='Etiqueta', how='right').to_excel('partes_interesse_stf.xlsx', index=False)